In [4]:
import time
zeitanfang = time.time()

In [ ]:
import pandas as pd
df = pd.read_csv("Exampledataframe.csv", sep=';', header=1)
df.drop(index=df.index[0], 
        axis=0, 
        inplace=True)
df = df.rename(columns={'answer_nps_comment': 'Review'})
df.head()

# Pre Processing

In [6]:
df.drop(['interview_id', 'answer_nps'], axis=1, inplace=True)

In [ ]:
#Create List of reviews
train_text_list = df['Review'].tolist()
print(train_text_list)

In [ ]:
#Tokenize the sentences
!pip3 install nltk
import nltk
from nltk import word_tokenize
from nltk import StanfordTagger
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

tagged_list = []
for i in train_text_list:
    tagged_text_list = nltk.word_tokenize(i)
    pos_tagged = nltk.pos_tag(tagged_text_list)
    tagged_list.append(pos_tagged)


print(tagged_list)

In [ ]:
#Filter the word with tag- noun,adjective,verb,adverb, number
def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
        for word,tag in text_list:
            if tag in ['CD','NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
                final_text.append(word)
        final_text_list.append(' '.join(final_text))
    return final_text_list

#train list after filter
final_tagged_list=filterTag(tagged_list)
print(final_tagged_list)

In [ ]:
data={'Review':final_tagged_list}
data = pd.DataFrame(data)
data.index = range(1,len(data)+1)
    
df["Review"] = data["Review"]
df.head(5)

# Training Aspect Extraction model

In [11]:
from sklearn.model_selection import train_test_split

X= df.Review
y = df.drop('Review',1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


#Only for Bagging
#df_train = X_train.to_frame().merge(y_train, how='inner', left_index=True, right_index=True)
#df_sample_train = df_train.sample(frac=1, replace=True, random_state=10)


#X_train = df_sample_train.Review
#y_train = df_sample_train.drop('Review',1)


X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train_aspect = y_train.fillna(10)
y_train_aspect = y_train_aspect.astype(str).astype(int)
y_train_aspect[y_train_aspect <= 0] = 1
y_train_aspect[y_train_aspect == 10] = 0

y_test_aspect = y_test.fillna(10)
y_test_aspect = y_test_aspect.astype(str).astype(int)
y_test_aspect[y_test_aspect <= 0] = 1
y_test_aspect[y_test_aspect == 10] = 0

import numpy as np
y_train_aspect = np.asarray(y_train_aspect, dtype=np.int64)
y_test_aspect = np.asarray(y_test_aspect, dtype=np.int64)

<ipython-input-11-5bfe9399528a>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  y = df.drop('Review',1)
<ipython-input-11-5bfe9399528a>:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  y_train = df_sample_train.drop('Review',1)


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
vect = CountVectorizer(max_df=1.0,stop_words='english')  
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

X_train_dtm = pd.DataFrame.sparse.from_spmatrix(X_train_dtm)
    
X_test_dtm = pd.DataFrame.sparse.from_spmatrix(X_test_dtm)   

In [13]:
y_train_aspect.shape

(1251, 14)

In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier


mlp = MLPClassifier().fit(X_train_dtm, y_train_aspect)



/home/moritz/anaconda3/envs/tf_gpu2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [19]:
#Boosting

from sklearn.ensemble import AdaBoostClassifier

import time
zeitanfang = time.time()

clf = OneVsRestClassifier(AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators=100, random_state=0))
clf.fit(X_train_dtm, y_train_aspect)

zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

Dauer Programmausführung:
191.43975734710693


/home/moritz/anaconda3/envs/tf_gpu2/lib/python3.9/site-packages/sklearn/multiclass.py:79: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(


In [18]:
#Predict the test data using classifiers
#y_pred_class_rfc = rfc.predict_proba(X_test_dtm)

y_pred_class_mlp = mlp.predict_proba(X_test_dtm)
#y_pred_class_boosting = clf.predict(X_test_dtm)

In [19]:
predictions2 = y_pred_class_mlp.tolist()
data2={'MLP_Predictions':predictions2}
data2 = pd.DataFrame(data2)

data2.to_csv('ac/mlp.csv', index=False)  
#data2.to_csv('bagging/ac/mlp.csv', index=False)  

In [ ]:
zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

# Training Aspect-based Sentiment Analysis model

In [16]:
#Create binary datasets for positive, negative and neutral sentiments
#Positive Dataset
X= df.Review
y = df.drop('Review',1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


#Only for Bagging
#df_train = X_train.to_frame().merge(y_train, how='inner', left_index=True, right_index=True)
#df_sample_train = df_train.sample(frac=2, replace=True, random_state=10)


#X_train = df_sample_train.Review
#y_train = df_sample_train.drop('Review',1)

#Train Dataset
df_positive_train_X = X_train
df_positive_train_y = y_train


df_positive_train_y = df_positive_train_y.fillna(0)
df_positive_train_y = df_positive_train_y.astype(str).astype(int)
df_positive_train_y[df_positive_train_y <= 0] = 0

#df_positive_train = df_positive_train_X.to_frame().merge(df_positive_train_y, how='inner', left_index=True, right_index=True)

df_positive_train_y["Review"] = df_positive_train_X
df_positive_train = df_positive_train_y

#Test Dataset
df_positive_test_X = X_test
df_positive_test_y = y_test

df_positive_test_y = df_positive_test_y.fillna(0)
df_positive_test_y = df_positive_test_y.astype(str).astype(int)
df_positive_test_y[df_positive_test_y <= 0] = 0

#df_positive_test = df_positive_test_X.to_frame().merge(df_positive_test_y, how='inner', left_index=True, right_index=True)

df_positive_test_y["Review"] = df_positive_test_X
df_positive_test = df_positive_test_y

/tmp/ipykernel_2333814/1330922059.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  y = df.drop('Review',1)


In [ ]:
#Get all aspects in one list
all_aspects = []
for col in y.columns:
    all_aspects.append(col)
print(all_aspects)

In [18]:
#Generating extra feature that indicates which aspect category is present in the review

def get_dict_aspect(y,all_aspects):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(all_aspects)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect



from sklearn.feature_extraction import DictVectorizer

#Creates a dictonionary of all aspects and a binary classifier which indicates if a aspect occour in a sentence.
train_dict_aspect=get_dict_aspect(y_train_aspect, all_aspects)

#Declare DictVectorizer
d_train=DictVectorizer()

X_train_aspect_dtm = d_train.fit_transform(train_dict_aspect)


#The same for the test dataset
test_dict_aspect=get_dict_aspect(y_test_aspect, all_aspects)
d_test=DictVectorizer()
X_test_aspect_dtm = d_test.fit_transform(test_dict_aspect)

In [19]:
#Function to classify a sentiment (Used for positive, negative)

def classify_sentiment(df_train,df_test,X_train_aspect_dtm,X_test_aspect_dtm):
    


    import numpy as np
    X_train = df_train.Review
    y_train = df_train.drop('Review',1)
    y_train = np.asarray(y_train, dtype=np.int64)

    X_test = df_test.Review
    y_test = df_test.drop('Review',1)
    y_test = np.asarray(y_test, dtype=np.int64)

    #convert sentences to vectors in order to be able to process them 
    vect_sen = CountVectorizer(stop_words='english',ngram_range=(1,2))  
    X_train_dtm = vect_sen.fit_transform(X_train)
    X_test_dtm = vect_sen.transform(X_test)
    #ombining word vector with extra feature. (Adds list of aspecs in sentences)
    from scipy.sparse import hstack
    X_train_dtm=hstack((X_train_dtm, X_train_aspect_dtm))
    X_test_dtm=hstack((X_test_dtm, X_test_aspect_dtm))
    
    X_train_dtm = pd.DataFrame.sparse.from_spmatrix(X_train_dtm)
    
    X_test_dtm = pd.DataFrame.sparse.from_spmatrix(X_test_dtm)   
    

    #Train models - OneVsRestClassifier: One Classifier per class will be trained
    C = 1.0 #SVregularization parameter
    #clf = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)).fit(X_train_dtm, y_train_aspect)
    mlp = OneVsRestClassifier(MLPClassifier()).fit(X_train_dtm,y_train)

    predictions = mlp.predict_proba(X_test_dtm)
    return (predictions)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

#For positive sentiment classifier
import time
zeitanfang = time.time()
predictions=classify_sentiment(df_positive_train,df_positive_test,X_train_aspect_dtm,X_test_aspect_dtm)

predictions = predictions.tolist()
data2={'MLP_Predictions':predictions}
data2 = pd.DataFrame(data2)
data3 = data2
data2.to_csv('asa/mlp_predictions_pos.csv', index=False)  
#data2.to_csv('bagging/asa/mlp_predictions_pos.csv', index=False)  

zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

In [20]:
import numpy as np
X_train = df_positive_train.Review
y_train = df_positive_train.drop('Review',1)
y_train = np.asarray(y_train, dtype=np.int64)

X_test = df_positive_test.Review
y_test = df_positive_test.drop('Review',1)
y_test = np.asarray(y_test, dtype=np.int64)

#convert sentences to vectors in order to be able to process them 
vect_sen = CountVectorizer(stop_words='english',ngram_range=(1,2))  
X_train_dtm = vect_sen.fit_transform(X_train).todense()
X_test_dtm = vect_sen.transform(X_test).todense()
#ombining word vector with extra feature. (Adds list of aspecs in sentences)
from scipy.sparse import hstack
X_train_dtm=hstack((X_train_dtm, X_train_aspect_dtm)).toarray()
X_test_dtm=hstack((X_test_dtm, X_test_aspect_dtm)).toarray()
    
#X_train_dtm = pd.DataFrame.sparse.from_spmatrix(X_train_dtm)
    
#X_test_dtm = pd.DataFrame.sparse.from_spmatrix(X_test_dtm)   

/tmp/ipykernel_2333814/2673588956.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  y_train = df_positive_train.drop('Review',1)
/tmp/ipykernel_2333814/2673588956.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  y_test = df_positive_test.drop('Review',1)


In [ ]:
#Boosting

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier

import time
zeitanfang = time.time()

class customMLPClassifer(MLPClassifier):
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled


    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and
                                            hasattr(self, "classes_")))

       

adabooster = OneVsRestClassifier(AdaBoostClassifier(base_estimator=customMLPClassifer(), n_estimators=25))

adabooster.fit(X_train_dtm,y_train_aspect)

zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

In [22]:
predictions = adabooster.predict(X_test_dtm)

predictions = predictions.tolist()
data2={'Boosting_Predictions':predictions}
data2 = pd.DataFrame(data2)
data3 = data2
data2.to_csv('boosting/mlp_predictions_boosting.csv', index=False)  